<a href="https://colab.research.google.com/github/fjadidi2001/DataScienceJourney/blob/master/Face_pretrainmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install inference-cli && inference server start

Error connecting to Docker daemon. Is docker installed and running? See https://www.docker.com/get-started/ for installation instructions.


In [3]:
from inference_sdk import InferenceHTTPClient

CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="2IGtFaicFMGaMwb2mX8A"
)

# Function to process multiple images
def detect_faces(image_paths):
    results = []
    for image_path in image_paths:
        result = CLIENT.infer(image_path, model_id="face-detection-mik1i/21")
        results.append(result)
    return results

# Example usage with 5 images
image_paths = [
    "/content/imgs/MultipleFaces.jpg",
    "/content/imgs/MultipleFaces2.jpg",
    "/content/imgs/SingleFace.jpg",
    "/content/imgs/SingleFace2.jpg",
    "/content/imgs/SingleFace3.jpg"
]

# Get detection results
detection_results = detect_faces(image_paths)

# Process and display results
for i, result in enumerate(detection_results):
    print(f"Image {i+1} detection results:")

    # If you want to extract face detection information
    if "predictions" in result:
        faces = result["predictions"]
        print(f"Found {len(faces)} faces")

        for j, face in enumerate(faces):
            confidence = face.get("confidence", 0)
            bbox = face.get("bbox", {})
            print(f"  Face {j+1}: Confidence {confidence:.2f}, Position: x={bbox.get('x')}, y={bbox.get('y')}, w={bbox.get('width')}, h={bbox.get('height')}")
    else:
        print("  No faces detected or error in detection")
    print()

Image 1 detection results:
Found 3 faces
  Face 1: Confidence 0.93, Position: x=None, y=None, w=None, h=None
  Face 2: Confidence 0.91, Position: x=None, y=None, w=None, h=None
  Face 3: Confidence 0.91, Position: x=None, y=None, w=None, h=None

Image 2 detection results:
Found 2 faces
  Face 1: Confidence 0.92, Position: x=None, y=None, w=None, h=None
  Face 2: Confidence 0.91, Position: x=None, y=None, w=None, h=None

Image 3 detection results:
Found 2 faces
  Face 1: Confidence 0.92, Position: x=None, y=None, w=None, h=None
  Face 2: Confidence 0.48, Position: x=None, y=None, w=None, h=None

Image 4 detection results:
Found 1 faces
  Face 1: Confidence 0.90, Position: x=None, y=None, w=None, h=None

Image 5 detection results:
Found 1 faces
  Face 1: Confidence 0.90, Position: x=None, y=None, w=None, h=None

